In [ ]:
import pandas as pd
import numpy as np
import time

avaliar os dados, algumas informações são incompletas para determinados anos
--> modelo 1: com variaveis para todos os anos~
--> modelo 2: complexo com as multiplas variaveis inseridas a partir de 2023

--> modelos podem ser extendidos para outros campeonatos

Currently we will block users sending requests to our sites more often than twenty requests in a minute.

https://www.sports-reference.com/bot-traffic.html

### Série B

In [ ]:
def coletar_estatisticas_serieb(id_squad, name_squad, ano):

  url_schedule = f"https://fbref.com/en/squads/{id_squad}/{ano}/matchlogs/c38/schedule/"
  time.sleep(4)
  url_shooting = f"https://fbref.com/en/squads/{id_squad}/{ano}/matchlogs/c38/shooting/"
  time.sleep(4)
  url_keeper = f"https://fbref.com/en/squads/{id_squad}/{ano}/matchlogs/c38/keeper/"
  time.sleep(4)
  url_misc = f"https://fbref.com/en/squads/{id_squad}/{ano}/matchlogs/c38/misc/"
  time.sleep(4)

  #Scores & Fixtures
  schedule = pd.read_html(url_schedule,header=0, attrs={'id':'matchlogs_for'})[0]
  schedule = schedule.drop(columns=['Match Report','Notes'])
  schedule['Round'] = schedule['Round'].str.replace("Matchweek", "")
  schedule['GDiff'] = schedule['GF'] - schedule['GA']
  schedule['idTeam'] = f"{id_squad}"
  schedule['Team'] = f"{name_squad}"
  schedule['Tournament'] = f"Campeonato Brasileiro - Série B - {ano}"
  schedule['Captain'] = schedule['Captain'].fillna("NA")
  schedule['Formation'] = schedule['Formation'].fillna("NA")
  schedule['Poss'] = schedule['Poss'].fillna(0)
  schedule['Attendance'] = schedule['Attendance'].fillna(0)
  schedule['Poss'] = schedule['Poss']/100
  schedule = schedule.astype({'Round': int, 'Poss':float}) #alterar as demais
  schedule['points'] = schedule['Result'].map({'W': 3, 'D': 1, 'L': 0})

  #Shooting
  shooting = pd.read_html(url_shooting,header=1, attrs={'id':'matchlogs_for'})[0]
  shooting = shooting[['Sh','SoT','SoT%','G/Sh','G/SoT','PK','PKatt']]
  shooting = shooting.fillna(0)
  shooting['SoT%'] = shooting['SoT%']/100
  shooting['PK%'] = shooting['PK']/shooting['PKatt']


  #Goalkeeping
  keeper = pd.read_html(url_keeper,header=1, attrs={'id':'matchlogs_for'})[0]
  keeper = keeper[['SoTA','Saves','Save%','CS','PKatt','PKsv']]
  keeper = keeper.rename(columns={'PKatt':'PKatt_A'})
  keeper = keeper.fillna(0)
  keeper['Save%'] = keeper['Save%']/100
  keeper['PKsv%'] = keeper['PKsv']/keeper['PKatt_A']


  #Miscellaneous
  misc = pd.read_html(url_misc,header=1, attrs={'id':'matchlogs_for'})[0]
  misc = misc[['CrdY','CrdR','2CrdY','Fls','Fld','Off','Crs','Int','TklW','OG']]
  misc = misc.fillna(0)

  new_df = pd.concat([schedule,shooting,keeper,misc],axis=1)
  new_df = new_df.fillna(0)

  #Ajustar os tipos de dados
  new_df[['Time','Round','Day','Venue','Result','Opponent','Captain','Formation','Referee','idTeam','Team','Tournament']] = new_df[['Time','Round','Day','Venue','Result','Opponent','Captain','Formation','Referee','idTeam','Team','Tournament']].astype(object)
  new_df[['GF','GA','Attendance','GDiff']] = new_df[['GF','GA','Attendance','GDiff']].astype(int)
  new_df[['Poss','SoT%','G/Sh','G/SoT','PK%','Save%','PKsv%']] = new_df[['Poss','SoT%','G/Sh','G/SoT','PK%','Save%','PKsv%']].astype(float)
  new_df['Date'] = pd.to_datetime(new_df['Date'], format='%Y-%m-%d')

  #Renomear as colunas
  novos_nomes = {'Date':'date','Time':'time','Round':'round','Day':'day','Venue':'venue','Result':'result','GF':'gf',\
  'GA':'ga','Opponent':'opponent','Poss':'possession','Attendance':'attendance','Captain':'captain',\
  'Formation':'formation','Referee':'referee','GDiff':'gdiff','idTeam':'idteam','Team':'team','Tournament':'tournament',\
  'Sh':'shots','SoT':'shotsontarget','SoT%':'p_shotsontarget','G/Sh':'goals_shots',\
  'G/SoT':'goals_shotsontarget','PK':'penalty_goals','PKatt':'penalty_attempted','PK%':'p_penaltyconverted',\
  'SoTA':'shotsontarget_against','Saves':'saves','Save%':'p_saves','CS':'cleansheets',\
  'PKatt_A':'penalty_attempted_against','PKsv':'penalty_saved','PKsv%':'p_penalty_saved',\
  'CrdY':'yellow_cards','CrdR':'red_cards','2CrdY':'two_yellow_cards',\
  'Fls':'fouls_commited','Fld':'fouls_draw','Off':'offsides','Crs':'crosses','Int':'interceptions','TklW':'tackles_won','OG':'own_goals'}

  new_df = new_df.rename(columns=novos_nomes)
  new_df = new_df[new_df['date'] > "1970-01-01"]#remover a linha total da tabela
  new_df['idgameteam'] = new_df['idteam'] + "_" + new_df['round'].astype(str) + "_" + new_df['tournament'].str[-4:]

  #Reorganizar as colunas
  new_df = new_df[['idgameteam','date','time','round','day','tournament','venue','idteam','team','opponent','result','points','gf','ga','possession','attendance','captain','formation','referee','gdiff','shots','shotsontarget','p_shotsontarget','goals_shots','goals_shotsontarget','penalty_goals','penalty_attempted','p_penaltyconverted','shotsontarget_against','saves','p_saves','cleansheets','penalty_attempted_against','penalty_saved','p_penalty_saved','yellow_cards','red_cards','two_yellow_cards','fouls_commited','fouls_draw','offsides','crosses','interceptions','tackles_won','own_goals']]
  time.sleep(8)

  return new_df

### Coletando Dados - Teste

In [ ]:
squads = {"7a1064a2":"Gremio-Novorizontino","3cc399a5":"Botafogo-SP"}

In [ ]:
#colunas reordenadas
df = pd.DataFrame(columns = ['idgameteam','date','time','round','day','tournament','venue','idteam','team','opponent','result','points','gf','ga','possession','attendance','captain','formation','referee','gdiff','shots','shotsontarget','p_shotsontarget','goals_shots','goals_shotsontarget','penalty_goals','penalty_attempted','p_penaltyconverted','shotsontarget_against','saves','p_saves','cleansheets','penalty_attempted_against','penalty_saved','p_penalty_saved','yellow_cards','red_cards','two_yellow_cards','fouls_commited','fouls_draw','offsides','crosses','interceptions','tackles_won','own_goals'])

for id_squad, name_squad in squads.items():
   new_df = coletar_estatisticas_serieb(id_squad, name_squad,"2023")
   df = pd.concat([df,new_df],ignore_index=True)

In [ ]:
df

,idgameteam,date,time,round,day,tournament,venue,idteam,team,opponent,...,yellow_cards,red_cards,two_yellow_cards,fouls_commited,fouls_draw,offsides,crosses,interceptions,tackles_won,own_goals
0,7a1064a2_1.0_2023,2023-04-15,16:00,1.0,Sat,Campeonato Brasileiro - Série B - 2023,Away,7a1064a2,Gremio-Novorizontino,Vila Nova,...,1,1,0,19,13,2,21,10,7,0
1,7a1064a2_2.0_2023,2023-04-22,17:00,2.0,Sat,Campeonato Brasileiro - Série B - 2023,Home,7a1064a2,Gremio-Novorizontino,Juventude,...,10,2,2,21,19,0,31,12,9,0
2,7a1064a2_3.0_2023,2023-04-30,15:30,3.0,Sun,Campeonato Brasileiro - Série B - 2023,Home,7a1064a2,Gremio-Novorizontino,Sport Recife,...,0,0,0,14,6,2,31,14,7,0
3,7a1064a2_4.0_2023,2023-05-03,19:00,4.0,Wed,Campeonato Brasileiro - Série B - 2023,Away,7a1064a2,Gremio-Novorizontino,Mirassol Futebol Clube,...,6,0,0,24,19,0,16,19,7,0
4,7a1064a2_5.0_2023,2023-05-06,20:30,5.0,Sat,Campeonato Brasileiro - Série B - 2023,Away,7a1064a2,Gremio-Novorizontino,Chapecoense,...,3,0,0,23,12,0,19,6,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,3cc399a5_34.0_2023,2023-10-27,21:30,34.0,Fri,Campeonato Brasileiro - Série B - 2023,Away,3cc399a5,Botafogo-SP,Guarani,...,2,0,0,14,15,1,14,7,7,0
72,3cc399a5_35.0_2023,2023-11-04,17:00,35.0,Sat,Campeonato Brasileiro - Série B - 2023,Home,3cc399a5,Botafogo-SP,Ceará,...,2,0,0,17,14,3,20,11,14,0
73,3cc399a5_36.0_2023,2023-11-11,17:00,36.0,Sat,Campeonato Brasileiro - Série B - 2023,Away,3cc399a5,Botafogo-SP,Chapecoense,...,1,1,0,11,19,2,13,7,8,0
74,3cc399a5_37.0_2023,2023-11-18,17:00,37.0,Sat,Campeonato Brasileiro - Série B - 2023,Away,3cc399a5,Botafogo-SP,Criciúma,...,1,0,0,15,10,0,3,7,6,0


In [ ]:
df.to_csv("df_teste.csv",index=False)

### Obtendo os ids e nomes dos times para cada temporada


In [ ]:
listas_times = pd.read_csv("lista_times_serieb.csv",encoding='latin1',sep=";")

l2016 = listas_times.loc[listas_times['Ano'] == 2016]
l2017 = listas_times.loc[listas_times['Ano'] == 2017]
l2018 = listas_times.loc[listas_times['Ano'] == 2018]
l2019 = listas_times.loc[listas_times['Ano'] == 2019]
l2020 = listas_times.loc[listas_times['Ano'] == 2020]
l2021 = listas_times.loc[listas_times['Ano'] == 2021]
l2022 = listas_times.loc[listas_times['Ano'] == 2022]
l2023 = listas_times.loc[listas_times['Ano'] == 2023]

lista_2016 = dict(zip(l2016['id'],l2016['Squad']))
lista_2017 = dict(zip(l2017['id'],l2017['Squad']))
lista_2018 = dict(zip(l2018['id'],l2018['Squad']))
lista_2019 = dict(zip(l2019['id'],l2019['Squad']))
lista_2020 = dict(zip(l2020['id'],l2020['Squad']))
lista_2021 = dict(zip(l2021['id'],l2021['Squad']))
lista_2022 = dict(zip(l2022['id'],l2022['Squad']))
lista_2023 = dict(zip(l2023['id'],l2023['Squad']))

In [ ]:
#exemplo de lista
lista_2021

{'d9fdd9d9': 'Botafogo-(RJ)',
 '78c617cc': 'Goiás',
 'd680d257': 'Coritiba',
 'f205258a': 'Avaí',
 '05aff519': 'CSA',
 '341d7cb0': 'Guarani',
 '7fcb6e83': 'CRB',
 'b5effd9e': 'Náutico',
 'f050c492': 'Vila-Nova',
 '83f55dbe': 'Vasco-da-Gama',
 'b162ebe7': 'Ponte-Preta',
 'd2dc922e': 'Operário',
 'b8daef43': 'Brusque-Futebol-Clube',
 '03ff5eeb': 'Cruzeiro',
 '740be0b0': 'Sampaio-Corrêa',
 '7886dd6c': 'Londrina',
 '2d4d7b6a': 'Remo',
 '33f95fe0': 'Vitória',
 'e48dfc07': 'Confiança',
 'd11febf7': 'Brasil-de-Pelotas'}

### Coletando os dados para cada temporada

In [ ]:
#2016
df_2016 = pd.DataFrame(columns = ['idgameteam','date','time','round','day','tournament','venue','idteam','team','opponent','result','points','gf','ga','possession','attendance','captain','formation','referee','gdiff','shots','shotsontarget','p_shotsontarget','goals_shots','goals_shotsontarget','penalty_goals','penalty_attempted','p_penaltyconverted','shotsontarget_against','saves','p_saves','cleansheets','penalty_attempted_against','penalty_saved','p_penalty_saved','yellow_cards','red_cards','two_yellow_cards','fouls_commited','fouls_draw','offsides','crosses','interceptions','tackles_won','own_goals'])

for id_squad, name_squad in lista_2016.items():
   new_df_2016 = coletar_estatisticas_serieb(id_squad, name_squad,"2016")
   df_2016 = pd.concat([df_2016,new_df_2016],ignore_index=True)

In [ ]:
#2017
df_2017 = pd.DataFrame(columns = ['idgameteam','date','time','round','day','tournament','venue','idteam','team','opponent','result','points','gf','ga','possession','attendance','captain','formation','referee','gdiff','shots','shotsontarget','p_shotsontarget','goals_shots','goals_shotsontarget','penalty_goals','penalty_attempted','p_penaltyconverted','shotsontarget_against','saves','p_saves','cleansheets','penalty_attempted_against','penalty_saved','p_penalty_saved','yellow_cards','red_cards','two_yellow_cards','fouls_commited','fouls_draw','offsides','crosses','interceptions','tackles_won','own_goals'])

for id_squad, name_squad in lista_2017.items():
   new_df_2017 = coletar_estatisticas_serieb(id_squad, name_squad,"2017")
   df_2017 = pd.concat([df_2017,new_df_2017],ignore_index=True)

In [ ]:
#2018
df_2018 = pd.DataFrame(columns = ['idgameteam','date','time','round','day','tournament','venue','idteam','team','opponent','result','points','gf','ga','possession','attendance','captain','formation','referee','gdiff','shots','shotsontarget','p_shotsontarget','goals_shots','goals_shotsontarget','penalty_goals','penalty_attempted','p_penaltyconverted','shotsontarget_against','saves','p_saves','cleansheets','penalty_attempted_against','penalty_saved','p_penalty_saved','yellow_cards','red_cards','two_yellow_cards','fouls_commited','fouls_draw','offsides','crosses','interceptions','tackles_won','own_goals'])

for id_squad, name_squad in lista_2018.items():
   new_df_2018 = coletar_estatisticas_serieb(id_squad, name_squad,"2018")
   df_2018 = pd.concat([df_2018,new_df_2018],ignore_index=True)

In [ ]:
#2019
df_2019 = pd.DataFrame(columns = ['idgameteam','date','time','round','day','tournament','venue','idteam','team','opponent','result','points','gf','ga','possession','attendance','captain','formation','referee','gdiff','shots','shotsontarget','p_shotsontarget','goals_shots','goals_shotsontarget','penalty_goals','penalty_attempted','p_penaltyconverted','shotsontarget_against','saves','p_saves','cleansheets','penalty_attempted_against','penalty_saved','p_penalty_saved','yellow_cards','red_cards','two_yellow_cards','fouls_commited','fouls_draw','offsides','crosses','interceptions','tackles_won','own_goals'])

for id_squad, name_squad in lista_2019.items():
   new_df_2019 = coletar_estatisticas_serieb(id_squad, name_squad,"2019")
   df_2019 = pd.concat([df_2019,new_df_2019],ignore_index=True)

In [ ]:
#2020
df_2020 = pd.DataFrame(columns = ['idgameteam','date','time','round','day','tournament','venue','idteam','team','opponent','result','points','gf','ga','possession','attendance','captain','formation','referee','gdiff','shots','shotsontarget','p_shotsontarget','goals_shots','goals_shotsontarget','penalty_goals','penalty_attempted','p_penaltyconverted','shotsontarget_against','saves','p_saves','cleansheets','penalty_attempted_against','penalty_saved','p_penalty_saved','yellow_cards','red_cards','two_yellow_cards','fouls_commited','fouls_draw','offsides','crosses','interceptions','tackles_won','own_goals'])

for id_squad, name_squad in lista_2020.items():
   new_df_2020 = coletar_estatisticas_serieb(id_squad, name_squad,"2020")
   df_2020 = pd.concat([df_2020,new_df_2020],ignore_index=True)

In [ ]:
#2021
df_2021 = pd.DataFrame(columns = ['idgameteam','date','time','round','day','tournament','venue','idteam','team','opponent','result','points','gf','ga','possession','attendance','captain','formation','referee','gdiff','shots','shotsontarget','p_shotsontarget','goals_shots','goals_shotsontarget','penalty_goals','penalty_attempted','p_penaltyconverted','shotsontarget_against','saves','p_saves','cleansheets','penalty_attempted_against','penalty_saved','p_penalty_saved','yellow_cards','red_cards','two_yellow_cards','fouls_commited','fouls_draw','offsides','crosses','interceptions','tackles_won','own_goals'])

for id_squad, name_squad in lista_2021.items():
   new_df_2021 = coletar_estatisticas_serieb(id_squad, name_squad,"2021")
   df_2021 = pd.concat([df_2021,new_df_2021],ignore_index=True)

In [ ]:
#2022
df_2022 = pd.DataFrame(columns = ['idgameteam','date','time','round','day','tournament','venue','idteam','team','opponent','result','points','gf','ga','possession','attendance','captain','formation','referee','gdiff','shots','shotsontarget','p_shotsontarget','goals_shots','goals_shotsontarget','penalty_goals','penalty_attempted','p_penaltyconverted','shotsontarget_against','saves','p_saves','cleansheets','penalty_attempted_against','penalty_saved','p_penalty_saved','yellow_cards','red_cards','two_yellow_cards','fouls_commited','fouls_draw','offsides','crosses','interceptions','tackles_won','own_goals'])

for id_squad, name_squad in lista_2022.items():
   new_df_2022 = coletar_estatisticas_serieb(id_squad, name_squad,"2022")
   df_2022 = pd.concat([df_2022,new_df_2022],ignore_index=True)

In [ ]:
#2023
df_2023 = pd.DataFrame(columns = ['idgameteam','date','time','round','day','tournament','venue','idteam','team','opponent','result','points','gf','ga','possession','attendance','captain','formation','referee','gdiff','shots','shotsontarget','p_shotsontarget','goals_shots','goals_shotsontarget','penalty_goals','penalty_attempted','p_penaltyconverted','shotsontarget_against','saves','p_saves','cleansheets','penalty_attempted_against','penalty_saved','p_penalty_saved','yellow_cards','red_cards','two_yellow_cards','fouls_commited','fouls_draw','offsides','crosses','interceptions','tackles_won','own_goals'])

for id_squad, name_squad in lista_2023.items():
   new_df_2023 = coletar_estatisticas_serieb(id_squad, name_squad,"2023")
   df_2023 = pd.concat([df_2023,new_df_2023],ignore_index=True)

In [ ]:
#unindo todos os dados em um unico dataframe
df_serieb = pd.concat([df_2016,df_2017,df_2018,df_2019,df_2020,df_2021,df_2022,df_2023],ignore_index=True)

In [ ]:
df_serieb.head()

,idgameteam,date,time,round,day,tournament,venue,idteam,team,opponent,...,yellow_cards,red_cards,two_yellow_cards,fouls_commited,fouls_draw,offsides,crosses,interceptions,tackles_won,own_goals
0,32d508ca_1.0_2016,2016-05-13,20:30,1.0,Fri,Campeonato Brasileiro - Série B - 2016,Away,32d508ca,Atl-Goianiense,Oeste,...,2,0,0,0,0,0,0,0,0,0
1,32d508ca_2.0_2016,2016-05-20,19:15,2.0,Fri,Campeonato Brasileiro - Série B - 2016,Home,32d508ca,Atl-Goianiense,Brasil (RS),...,0,0,0,0,0,0,0,0,0,0
2,32d508ca_3.0_2016,2016-05-24,19:15,3.0,Tue,Campeonato Brasileiro - Série B - 2016,Away,32d508ca,Atl-Goianiense,Ceará,...,4,1,1,0,0,0,0,0,0,0
3,32d508ca_4.0_2016,2016-05-28,16:00,4.0,Sat,Campeonato Brasileiro - Série B - 2016,Home,32d508ca,Atl-Goianiense,Vila Nova,...,3,0,0,0,0,0,0,0,0,0
4,32d508ca_5.0_2016,2016-05-31,20:30,5.0,Tue,Campeonato Brasileiro - Série B - 2016,Away,32d508ca,Atl-Goianiense,Luverdense,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
df_serieb.to_csv("df_serieb.csv",index=False) #exportando para csv